In [1]:
# Instalación de librerías
!pip install transformers torch scikit-learn nltk

In [2]:
# Importaciones
import pandas as pd
import numpy as np
import torch
import nltk
from google.colab import drive
from transformers import AutoTokenizer, AutoModel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
import re

In [3]:
# Montar Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Descargar recursos NLTK
#nltk.download('punkt')
#nltk.download('stopwords')
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_rus to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |  

True

In [5]:
class ChatbotAvanzado:
    def __init__(self, ruta_dataset):
        # Cargar dataset desde Drive
        self.df = pd.read_csv(ruta_dataset)

        # Configuración de procesamiento
        self.stemmer = SnowballStemmer('spanish')  #Para reducir palabras a su raíz
        self.stop_words = set(stopwords.words('spanish')) #Palabras comunes que se ignoran

        # Cargar modelo de lenguaje "BERT multilingüe pre-entrenado para procesar lenguaje natural".
        self.tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
        self.model = AutoModel.from_pretrained("bert-base-multilingual-cased")

        # Preprocesar datos
        self.preprocesar_datos()

        # Vectorización
        self.vectorizador = TfidfVectorizer(
            stop_words=list(self.stop_words),
            max_features=5000
        )
        self.X = self.vectorizador.fit_transform(self.df['texto_limpio'])

        # Contexto de conversación
        self.contexto = []

    def preprocesar_texto(self, texto): #Este método limpia y preprocesa el texto
        """Preprocesar texto"""
        texto = str(texto).lower()
        texto = re.sub(r'[^a-záéíóúñ\s]', '', texto)
        tokens = word_tokenize(texto)
        tokens = [self.stemmer.stem(word) for word in tokens if word not in self.stop_words]
        return ' '.join(tokens)

    def preprocesar_datos(self): #Aplica el preprocesamiento a la columna 'pregunta' del dataset.
        """Preprocesar columnas del dataset"""
        self.df['texto_limpio'] = self.df['pregunta'].apply(self.preprocesar_texto)

    def obtener_embedding_bert(self, texto): #Representaciones vectoriales del texto.
        """Obtener embedding con BERT"""
        inputs = self.tokenizer(texto, return_tensors="pt", padding=True, truncation=True, max_length=512)
        with torch.no_grad():
            outputs = self.model(**inputs)
        return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

    def buscar_respuesta_semantica(self, consulta): #Busca la respuesta más relevante combinando similitud TF-IDF y similitud basada en BERT.
        """Buscar respuesta usando similitud semántica"""
        consulta_limpia = self.preprocesar_texto(consulta)

        # Vectorización TF-IDF
        consulta_vectorizada = self.vectorizador.transform([consulta_limpia])
        similitudes_tfidf = cosine_similarity(consulta_vectorizada, self.X)[0]

        # Embedding BERT para similitud semántica
        consulta_embedding = self.obtener_embedding_bert(consulta)

        # Combinar métodos de similitud
        indices_top = np.argsort(similitudes_tfidf)[::-1][:5]

        mejores_respuestas = []
        for idx in indices_top:
            respuesta_candidata = self.df.iloc[idx]
            similitud_bert = cosine_similarity(
                [consulta_embedding],
                [self.obtener_embedding_bert(respuesta_candidata['pregunta'])]
            )[0][0]

            mejores_respuestas.append({
                'respuesta': respuesta_candidata['respuesta'],
                'similitud_tfidf': similitudes_tfidf[idx],
                'similitud_bert': similitud_bert
            })

        # Ordenar por una combinación de similitudes
        mejores_respuestas.sort(key=lambda x: (x['similitud_tfidf'] + x['similitud_bert']), reverse=True)

        return mejores_respuestas[0]['respuesta'] if mejores_respuestas else "Lo siento, no puedo encontrar una respuesta adecuada."

    def manejar_contexto(self, consulta): #Mantiene un contexto de las últimas 3 consultas y busca una respuesta basada en este contexto.
        """Manejar contexto de conversación"""
        self.contexto.append(consulta)
        if len(self.contexto) > 3:
            self.contexto.pop(0)

        return self.buscar_respuesta_semantica(consulta)

    def iniciar_chat(self): #Inicia una interfaz de chat en la consola, permitiendo al usuario interactuar con el chatbot.
        """Iniciar chatbot interactivo"""
        print("🤖 Chatbot de Servicio al Cliente")
        print("Escribe 'salir' para terminar la conversación\n")

        while True:
            consulta = input("👤 Tú: ")

            if consulta.lower() == 'salir':
                print("🤖 Gracias por usar nuestro servicio. ¡Hasta luego!")
                break

            respuesta = self.manejar_contexto(consulta)
            print(f"🤖 Respuesta: {respuesta}\n")

# Ruta del dataset en Google Drive
RUTA_DATASET = '/content/drive/My Drive/LLM/datos_chatbot_soporte_tecnico.csv'


In [10]:
# Inicializar y ejecutar chatbot
chatbot = ChatbotAvanzado(RUTA_DATASET)
chatbot.iniciar_chat()

🤖 Chatbot de Servicio al Cliente
Escribe 'salir' para terminar la conversación

👤 Tú: tipos de baterias
🤖 Respuesta: Ion-Litio más comunes, Polímero de Litio más delgadas, Níquel menos eficientes.

👤 Tú: deseo saber las caracteristicas del modelo X
🤖 Respuesta: El modelo X cuenta con una pantalla de 6.5 pulgadas, procesador Snapdragon 888, 8 GB de RAM, y una batería de 4500 mAh.

👤 Tú: ¿Cómo extender vida batería laptop?
🤖 Respuesta: Mantén carga entre 20-80%, evita temperaturas extremas, usa modo ahorro energía.

👤 Tú: Mi smartphone no enciende
🤖 Respuesta: Intenta mantener presionado el botón de encendido durante 10 segundos. Si no responde, conecta el cargador y espera unos minutos antes de intentar encenderlo nuevamente.

👤 Tú: sigue sin encender, que hago?
🤖 Respuesta: Si presionar el botón de encendido no funciona, conecta el smartphone al cargador original y déjalo cargando durante al menos 15 minutos antes de intentar encenderlo nuevamente.

👤 Tú: ¿Cuál es la garantía de mi lap